In [1]:

import shutil


def clean_project():
    try:
        shutil.rmtree('spark-warehouse')
        shutil.rmtree('metastore_db')
        shutil.rmtree('speed_test/_checkpoint')
    except: 
        pass
        
#clean_project()

In [2]:
from delta import *
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [9]:
sql = lambda statement,limit=5 : spark.sql(statement).limit(limit).toPandas()

In [3]:
minutes_to_seconds = lambda x: x * 60
run_every_minutes = minutes_to_seconds(10)

In [4]:

MAIN_PATH = 'speed_test/'
TEMP_PATH = f'{MAIN_PATH}temp/'
WORKING_PATH= f'{MAIN_PATH}raw_data/'
CHECK_POINT = f'{MAIN_PATH}_checkpoint'

SCHEMA = 'speed_test'
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {SCHEMA} ")

spark.sql(
    "set spark.databricks.delta.changeDataFeed.timestampOutOfRange.enabled = true;"
)
spark.sql("set SQLConf.ADAPTIVE_EXECUTION_ENABLED.key= true")

BRONZE_TABLE = f'{SCHEMA}.speed_test_logs'
spark.sql(f"""
  CREATE EXTERNAL TABLE IF NOT EXISTS {BRONZE_TABLE} 
  (log_timestamp Timestamp
  ) USING delta
  LOCATION '{WORKING_PATH.replace('speed_test','speed_test.db')}'
  TBLPROPERTIES (delta.enableChangeDataFeed = true)
""")

24/04/14 20:22:27 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:22:27 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:22:28 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
24/04/14 20:22:28 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore hadoop@127.0.1.1


DataFrame[]

In [11]:
sql(f'DESCRIBE HISTORY  {BRONZE_TABLE}')

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2,2024-04-14 20:13:04.146,None,None,STREAMING UPDATE,"{'epochId': '1', 'outputMode': 'Append', 'quer...",None,None,None,1.0,Serializable,True,"{'numOutputRows': '1', 'numRemovedFiles': '0',...",None,Apache-Spark/3.5.0 Delta-Lake/3.1.0
1,1,2024-04-14 20:12:48.526,None,None,STREAMING UPDATE,"{'epochId': '0', 'outputMode': 'Append', 'quer...",None,None,None,0.0,Serializable,True,"{'numOutputRows': '57', 'numRemovedFiles': '0'...",None,Apache-Spark/3.5.0 Delta-Lake/3.1.0
2,0,2024-04-14 20:11:09.046,None,None,CREATE TABLE,"{'description': None, 'partitionBy': '[]', 'pr...",None,None,None,NaN,Serializable,True,{},None,Apache-Spark/3.5.0 Delta-Lake/3.1.0


In [5]:
streaming_logs_schema = StructType([StructField("download",
                    StructType([StructField("bandwidth", LongType(), True),
                                StructField("bytes", LongType(), True),
                                StructField("elapse", LongType(), True),
                                StructField("latency", StringType(), True)])
                                      , True),
             StructField("interface", StringType(), True),
             StructField("isp",StructType(), True),
             StructField("ping", StructType(), True),
             StructField("result", StructType([StructField("id", StringType(), True),
                                StructField("url", StringType(), True),
                                StructField("persisted", BooleanType(), True),
                               ]), True),
             StructField("server", StringType(), True),
             StructField("timestamp", TimestampType(), True),
             StructField("type", StringType(), True),
             StructField("upload", 

                    StructType([StructField("bandwidth", LongType(), True),
                                StructField("bytes", LongType(), True),
                                StructField("elapse", LongType(), True),
                                StructField("latency", StringType(), True)]), True)
                                ])


streaming_logs_sdf = spark.readStream.schema(streaming_logs_schema)\
    .json(WORKING_PATH)\
            .where(F.col('timestamp').isNotNull())\
            .select([F.col('result').getItem('id').alias('test_id'),
                F.col("timestamp").alias('log_timestamp'),
                F.to_date('timestamp').alias('date'),
                F.hour('timestamp').alias('hour'),
                F.col('download').getItem('bytes').alias('download_bytes'),
                F.col('upload').getItem('bytes').alias('upload_bytes')])\
    .withColumn('part_of_the_day',
                (F.when((F.col('hour')>=5) & (F.col('hour')<12),'Morning')
                     .when((F.col('hour')>=12) & (F.col('hour')<17),'Afternoon')
                     .when((F.col('hour')>=17) &( F.col('hour')<21),'Evening')
                     .otherwise('Night')))\
    .withColumn('download_Mbytes',F.col('download_bytes')/1000000)\
        .withColumn('upload_Mbytes',F.col('upload_bytes')/1000000)\
        .withColumn('log_file',F.input_file_name())


# Change data feed
Change Data Feed (CDF) feature allows Delta tables to track row-level changes between versions of a Delta table. When enabled on a Delta table, the runtime records “change events” for all the data written into the table. This includes the row data along with metadata indicating whether the specified row was inserted, deleted, or updated.

You can read the change events in batch queries using DataFrame APIs (that is, df.read) and in streaming queries using DataFrame APIs (that is, df.readStream).

Use cases
Change Data Feed is not enabled by default. The following use cases should drive when you enable the change data feed.

Silver and Gold tables: Improve Delta performance by processing only row-level changes following initial MERGE, UPDATE, or DELETE operations to accelerate and simplify ETL and ELT operations.

Transmit changes: Send a change data feed to downstream systems such as Kafka or RDBMS that can use it to incrementally process in later stages of data pipelines.

Audit trail table: Capture the change data feed as a Delta table provides perpetual storage and efficient query capability to see all changes over time, including when deletes occur and what updates were made.



|Column name|Type| Values  |  
|---|---|---|
| _change_type  | String  | insert, update_preimage , update_postimage, delete  | 
|  _commit_version | Long  |  The Delta log or table version containing the change. |  
|  _commit_timestamp | Timestamp  | The timestamp associated when the commit w

## Use cases
Change Data Feed is not enabled by default. The following use cases should drive when you enable the change data feed.

Silver and Gold tables: Improve Delta performance by processing only row-level changes following initial MERGE, UPDATE, or DELETE operations to accelerate and simplify ETL and ELT operations.

Transmit changes: Send a change data feed to downstream systems such as Kafka or RDBMS that can use it to incrementally process in later stages of data pipelines.

Audit trail table: Capture the change data feed as a Delta table provides perpetual storage and efficient query capability to see all changes over time, including when deletes occur and what updates were ## Enable change data feed

- New table: Set the table property delta.enableChangeDataFeed = true in the CREATE TABLE command:<br>```CREATE TABLE student (id INT, name STRING, age INT) TBLPROPERTIES (delta.enableChangeDataFeed = true)```
  
- Existing table: Set the table property delta.enableChangeDataFeed = true in the ALTER TABLE command.<br>```ALTER TABLE myDeltaTable SET TBLPROPERTIES (delta.enableChangeDataFeed = true)```

- All new tables:<br>
      ```set spark.databricks.delta.properties.defaults.enableChangeDataFeed = true```made.

as created.  |

In [34]:
# version as ints or longs
spark.read.format("delta") \
  .option("readChangeFeed", "true") \
  .option("startingVersion", 0) \
  .option("endingVersion", 10) \
  .table(BRONZE_TABLE).limit(5).toPandas()

,log_timestamp,test_id,date,hour,download_bytes,upload_bytes,part_of_the_day,download_Mbytes,upload_Mbytes,log_file,_change_type,_commit_version,_commit_timestamp
0,2024-04-13 19:29:26,65e2e801-8747-4c18-93cf-a9a7dc013706,2024-04-13,19,443247280,12154512,Evening,443.247280,12.154512,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
1,2024-04-13 19:47:06,7c4cdd5a-7030-45c7-8955-d879e1114ef5,2024-04-13,19,499868424,7509328,Evening,499.868424,7.509328,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
2,2024-04-13 19:54:04,7f430e05-638b-4de7-8f12-ad68cedb259f,2024-04-13,19,444856008,6773744,Evening,444.856008,6.773744,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
3,2024-04-13 19:56:33,4b04c0ff-f5f0-4967-808d-76900bbd75f0,2024-04-13,19,371986856,7849608,Evening,371.986856,7.849608,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
4,2024-04-14 14:43:42,48b17f3e-0dbf-41cd-920d-c73a1d6c94b9,2024-04-14,14,543485080,6770848,Afternoon,543.485080,6.770848,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526


In [36]:
streaming_logs_sdf.writeStream.format('delta')\
.outputMode("append")\
.option("checkpointLocation", CHECK_POINT)\
.trigger(processingTime=f'{run_every_minutes} seconds')\
.option("overwriteSchema", "true")\
.option("mergeSchema", "true")\
.toTable(BRONZE_TABLE)

24/04/14 20:51:29 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/14 20:51:29 WARN StreamingQueryManager: Stopping existing streaming query [id=79a06c0d-5997-4738-aaca-4b98ff67b6fb, runId=c71339e7-61a7-4bdb-ba92-1fd2ae94379f], as a new run is being started.


24/04/14 20:51:36 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:51:36 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:51:46 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:51:46 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:51:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:51:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:52:06 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:52:06 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [13]:
import subprocess, sys
import threading
import time
from datetime import datetime as dt
import os
from glob import glob
os.makedirs(WORKING_PATH,exist_ok=True)


def speed_test():
    
    def move_temp_to_raw():
        files = glob(f'{TEMP_PATH}*.json')
        list(map(lambda x: os.rename(x, x.replace(TEMP_PATH,WORKING_PATH)),files ))
        
    while True:
        now = dt.now()
        file_name =  f"log_{now.strftime('%Y_%m_%d_%H_%M_%s')}.json"
        subprocess.run(f'touch {TEMP_PATH}{file_name} && speedtest --format=json >> {TEMP_PATH}{file_name} && mv  {TEMP_PATH}{file_name} {WORKING_PATH}{file_name}', shell = True, executable="/bin/bash")
        move_temp_to_raw()
        time.sleep(run_every_minutes)
        


thread = threading.Thread(target=speed_test)
thread.start()


24/04/14 20:12:48 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
24/04/14 20:12:48 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:12:48 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [38]:
sql(f'''SELECT * FROM table_changes('{BRONZE_TABLE}',0) LIMIT 5''')

,log_timestamp,test_id,date,hour,download_bytes,upload_bytes,part_of_the_day,download_Mbytes,upload_Mbytes,log_file,_change_type,_commit_version,_commit_timestamp
0,2024-04-13 19:29:26,65e2e801-8747-4c18-93cf-a9a7dc013706,2024-04-13,19,443247280,12154512,Evening,443.247280,12.154512,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
1,2024-04-13 19:47:06,7c4cdd5a-7030-45c7-8955-d879e1114ef5,2024-04-13,19,499868424,7509328,Evening,499.868424,7.509328,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
2,2024-04-13 19:54:04,7f430e05-638b-4de7-8f12-ad68cedb259f,2024-04-13,19,444856008,6773744,Evening,444.856008,6.773744,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
3,2024-04-13 19:56:33,4b04c0ff-f5f0-4967-808d-76900bbd75f0,2024-04-13,19,371986856,7849608,Evening,371.986856,7.849608,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526
4,2024-04-14 14:43:42,48b17f3e-0dbf-41cd-920d-c73a1d6c94b9,2024-04-14,14,543485080,6770848,Afternoon,543.485080,6.770848,file:///home/hadoop/deltalake/speed_test/raw_d...,insert,1,2024-04-14 20:12:48.526


24/04/14 20:52:26 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:52:26 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:52:36 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:52:36 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:52:46 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:52:46 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:52:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:52:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:53:06 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:53:06 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:53:16 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:53:16 WAR

In [15]:
plot_bgcolor = "#2c292d"
#paper_bgcolor ="#211f22"
paper_bgcolor ="#1a1d21"
download_color = "#ab9df2"
upload_color = "#78dce8"
default_fontcolor = "white"

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from dash.dependencies import Input, Output
from dash import dcc, html
import dash_bootstrap_components as dbc
import dash


def line_chart_download_vs_upload(fig,df):

    x = df.log_timestamp


    
    fig.add_trace(
        go.Scatter(
            x=x, y=df.upload_Mbytes, name="Upload (Mbps)", line=dict(color=upload_color)
        ),row =1, col=2,
    )
    fig.update_yaxes(
    title_text="<b>Upload (Mbps)</b>",
    color=upload_color,
    rangemode = 'tozero',
     showgrid=False,
    row =1, col=2,
    )



    fig.add_trace(
        go.Scatter(
            x=x,
            y=df.download_Mbytes,
            name="Download (Mbps)",
            line=dict(color=download_color),

        ),row =2, col=2,
    )

    fig.update_yaxes(
        title_text="<b>Download (Mbps)</b>",
        color=download_color,
        rangemode = 'tozero',
         showgrid=False,
        row =2, col=2,
    )

    fig.update_xaxes(
        showgrid=False,
    )

    fig.update_layout(
        plot_bgcolor=plot_bgcolor,
        paper_bgcolor=paper_bgcolor,
        font=dict(color="white"),
        legend=dict(orientation="h", yanchor="bottom", y=-0.15, xanchor="right", x=1),
    )
   
   


def gauges_indicators(fig,value):

    def gauge_chart(value, steps, title, color):
        max_step = steps[-1][-1]
        title = f"{title} <span style='font-size:0.8em;color:gray'>MBps</span><br><span style='font-size:0.5em;color:gray'>Average</span>"
        gauge = go.Indicator(
                mode="gauge+number+delta",
                value=value,
                domain={"x": [0.25,0.55], "y":[0.25,0.55]},
                title={"text": title, "font": {"size": 25},'align': 'center' },
                delta={"reference": steps[-1][0],  "font": {"size": 13},"increasing": {"color": color}},
                number={"font": {"size": 25}},
                gauge={
                "axis": {
                    "range": [None, max_step],
                    "tickwidth": 2,
                    "tickcolor": plot_bgcolor,
                },
                "bar": {"color": color},
                "bgcolor": "white",
                "borderwidth": 2,
                "bordercolor": "gray",
                "steps": [
                    {"range": steps[0], "color": "#ff6188"},
                    {"range": steps[1], "color": "#fc9867"},
                    {"range": steps[2], "color": "#a9dc76"},
                ],
            },
            )
        
       
        return gauge

    
    fig.add_trace( gauge_chart(
        value["upload_Mbytes"],
        steps=[[0, 10], [10, 15], [15, 20]],
        title=f"<span style='font-size:0.8em;color:{upload_color}'>Upload</span>",
        color=upload_color,
    ),row =1, col=1)

    fig.add_trace(gauge_chart(
        value["download_Mbytes"],
        steps=[[0, 450], [450, 600], [600, 700]],
        title=f"<span style='font-size:0.8em;color:{download_color}'>Download</span>",
        color=download_color,
    )  ,row =2, col=1 )

    fig.update_layout(
            paper_bgcolor=paper_bgcolor,
            font={"color": "white", "family": "Arial"},
            showlegend=False
        )
    fig.update_traces(
        number=dict(font=dict(size=28)),
        delta=dict(font=dict(size=25))
        )

def multiplot_speedtest(df):

    fig = make_subplots(
    rows=2, cols=2,
    specs=[
            [{"type": "domain"}, {} ],
            [ {"type": "domain"}, {}]
    ],
    column_widths = [0.30,0.70],
    row_heights = [0.25,0.25],
    horizontal_spacing=0.15,
    vertical_spacing=0.15,
          )
    
    values = df.iloc[-3:].mean()
    gauges_indicators(fig,values)
    line_chart_download_vs_upload(fig,df)
    fig.update_layout(height=550, margin=dict(l=35,r=35,b=30,t=55))
    
    return fig

    

def register_Callback(app):
    @app.callback(
        Output("stream_line_chart", "figure"),
        [Input("interval-component", "n_intervals"),
        ],
    )
    def streamFig(intervals):
        df = spark.read.table(BRONZE_TABLE).where(F.col('log_timestamp')  >= (F.current_timestamp() - F.expr("INTERVAL 60 minutes")) ).orderBy('log_timestamp',ascending=False).limit(10).toPandas().sort_values('log_timestamp',ascending=True)
        return multiplot_speedtest(df)



config = {"displaylogo": False, "scrollZoom": False, "displayModeBar": False}

updates = dcc.Interval(
    id="interval-component", interval=10000, n_intervals=0  # in milliseconds
)


navbar = dbc.Navbar(
    dbc.Container(
        [
            html.A(
                # Use row and col to control vertical alignment of logo / brand
                dbc.Row(
                    [
                        dbc.Col(html.Img(src="https://www.pinclipart.com/picdir/big/491-4917274_panama-flag-png-palestine-flag-vector-clipart.png", height="30px")),
                        dbc.Col(dbc.NavbarBrand("Network Speed Test by Jose Quesada", className="ms-2")),
                    ],
                    align="center",
                    className="g-0",
                ),
                href="https://plotly.com",
                style={"textDecoration": "none"},
            ),
            dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
            
        ]
    ),
    color=paper_bgcolor,
    dark=True,
)


streaming_col = dbc.Col(dcc.Graph(id="stream_line_chart", config=config))

layout = dbc.Container(
    
    [ navbar,
    dbc.Container([updates,
                    dcc.Store(id='last_32hrs'),
                                dbc.Row(streaming_col), 
    ],
            style={
                "background-color": paper_bgcolor,
                "color": default_fontcolor
            })])

app = dash.Dash(
    external_stylesheets=[
        "https://cdn.jsdelivr.net/npm/bootstrap@4.3.1/dist/css/bootstrap.min.css"
    ],
)
#app.config.suppress_callback_exceptions = True
app.layout = layout
register_Callback(app)
app.run()

24/04/14 20:33:14 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:33:14 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
[2024-04-14 20:33:15,044] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/hadoop/.local/lib/python3.10/site-packages/flask/app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
  File "/home/hadoop/.local/lib/python3.10/site-packages/flask/app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/home/hadoop/.local/lib/python3.10/site-packages/flask/app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
  File "/home/hadoop/.local/lib/python3.10/site-packages/flask/app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
  File "/home/hadoop/.local/lib/python3.10/site-packages/dash/dash.p

In [13]:
print(1)

1


24/04/14 20:32:46 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:32:46 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:32:46 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
24/04/14 20:32:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:32:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
24/04/14 20:33:07 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
24/04/14 20:33:07 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [ ]:
from delta.tables import *

deltaTableLogs = DeltaTable.forName(spark, BRONZE_TABLE )

In [ ]:
windowedCounts = words \
    .withWatermark("timestamp", "10 minutes") \
    .groupBy(
        window(words.timestamp, "10 minutes", "5 minutes"),
        words.word) \
    .count()

In [ ]:



dfUpdates = deltaTablePeopleUpdates.toDF()

deltaTablePeople.alias('people') \
  .merge(
    dfUpdates.alias('updates'),
    'people.id = updates.id'
  ) \
  .whenMatchedUpdate(set =
    {
      "id": "updates.id",
      "firstName": "updates.firstName",
      "middleName": "updates.middleName",
      "lastName": "updates.lastName",
      "gender": "updates.gender",
      "birthDate": "updates.birthDate",
      "ssn": "updates.ssn",
      "salary": "updates.salary"
    }
  ) \
  .whenNotMatchedInsert(values =
    {
      "id": "updates.id",
      "firstName": "updates.firstName",
      "middleName": "updates.middleName",
      "lastName": "updates.lastName",
      "gender": "updates.gender",
      "birthDate": "updates.birthDate",
      "ssn": "updates.ssn",
      "salary": "updates.salary"
    }
  ) \
  .execute()